<img src="Imagenes/Mac_wallpaper_3.png" width="50%">

# Creando los primeros circuitos

Ahora que ya conoces las compuertas cuánticas más básicas es momento de poner ese conocimiento en uso con algunos ejemplos sencillos.

### Compuertas controladas

La compuerta CNOT no es la única compuerta controlada que se puede implementar. De echo, en Qiskit podemos implementar fácilmente una compuerta $Z$ controlada, conocida como la compuerta $CZ$. Esta, como su nombre indica, implementa una compuerta $Z$ en el qubit objetivo solo si el qubit control está en el estado $|1\rangle$.

Su sintaxis es `.cz(control,objetivo)`.

In [ ]:
from qiskit import QuantumCircuit, execute, Aer

qc = QuantumCircuit(2)

#Aplicamos la compuerta cz con el qubit cero como el qubit control
qc.cz(0,1)

qc.draw(output="mpl")

Notamos que, a diferencia de la compuerta CNOT, en este caso el diagrama no parece distinguir entre el qubit control y el objetivo. Esto se debe a que si nos ponemos a pensar, esta compuerta afecta únicamente al estado $|11\rangle$, pues si bien el estado $|10\rangle$ "activa" el operador $Z$ sobre el qubit cero, este no se ve alterado de ninguna forma pues el operador $Z$ solo afecta al estado $|1\rangle$. Comprobemos esto con el simulador.

In [ ]:
from qiskit.visualization import array_to_latex

#Creamos una lista para almacenar las cuatro posibles combinaciones de dos qubits
estados = ["00","01","10","11"]

for estado in estados:
#Creamos un circuito con dos qubits sin bits clásicos, pues no vamos a hacer mediciones
    qc = QuantumCircuit(2)
    
#Preparamos nuestro estado inicial en base al estado seleccionado
    if int(estado[0]) == 1:
        qc.x(1)
    if int(estado[1]) == 1:
        qc.x(0)

#Aplicamos el operador CZ a cada estado base 
    qc.cz(0,1)

#Ejecutamos el simulador y obtenemos el vector de estado 
    job = execute(qc,Aer.get_backend('statevector_simulator'),optimization_level=0)
    precision = 3 
    vector_estado = job.result().get_statevector(qc,precision).data
#Imprimimos nuestros resultados
    display(array_to_latex(vector_estado, prefix="CZ|"+estado+"\\rangle ="))

Si bien podemos implementar esta compuerta directamente en el simulador, este no es el caso en hardware real. <br>
Las computadoras cuánticas no pueden implementar todas las compuertas que podemos crear teóricamente, y las compuertas que distintos procesadores pueden implementar directamente varían en cada caso. Las compuertas que se pueden aplicar directamente son conocidas como las *compuertas físicas*, y generalmente son un grupo de compuertas de un qubit, y una compuerta de dos qubits. La compuerta de dos qubits por excelencia es la compuerta CNOT, por lo que cualquier otra compuerta de dos qubits generalmente es descompuesta en varias compuertas de un qubit y CNOTs antes de ser implementada. Entonces ¿Cómo podríamos implementar una compuerta $CZ$ con estas condiciones? Bueno, veamos la siguiente operación

$\hspace{2 cm} HXH = \begin{pmatrix} \frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} & -\frac{1}{\sqrt{2}} \end{pmatrix}\begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}\begin{pmatrix} \frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} & -\frac{1}{\sqrt{2}} \end{pmatrix} = \begin{pmatrix} \frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} & -\frac{1}{\sqrt{2}} \end{pmatrix}\begin{pmatrix} \frac{1}{\sqrt{2}} & -\frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \end{pmatrix} = \begin{pmatrix}1 & 0 \\ 0 & -1\end{pmatrix} = Z$

De esta forma, vemos que si colocamos compuertas Hadamard en el qubit objetivo antes y después de la compuerta CNOT, estaremos aplicando una compuerta $Z$ sobre el qubit objetivo en lugar de una compuerta $X$. Comprobémoslo con el simulador. 

In [ ]:
qc = QuantumCircuit(2)

qc.h(1)
qc.cx(0,1)
qc.h(1)

qc.draw(output="mpl")

In [ ]:
estados = ["00","01","10","11"]

for estado in estados:

    qc = QuantumCircuit(2)
    
    if int(estado[0]) == 1:
        qc.x(1)
    if int(estado[1]) == 1:
        qc.x(0)

    qc.h(1)    
    qc.cx(0,1)
    qc.h(1)

    job = execute(qc,Aer.get_backend('statevector_simulator'),optimization_level=0)
    precision = 3 
    vector_estado = job.result().get_statevector(qc,precision).data

    display(array_to_latex(vector_estado, prefix="CZ|"+estado+"\\rangle ="))

*Nota: Este es un ejemplo con fines didácticos. La verdadera descomposición de una compuerta $CZ$ dependerá de las compuertas de un qubit que sean parte de las compuertas físicas de un procesador específico.*

Qiskit cuenta con más operadores controlados, pero para este taller bastará con solo conocer los que hemos visto.

### Implementar compuertas clásicas

Como ya hemos visto, es posible implementar un análogo las compuertas clásicas NOT y CNOT de manera directa en Qiskit. Sin embargo, también es posible implementar un análogo de otras compuertas clásicas, pero en este caso deberemos construirlas. Empecemos con una compuerta sencilla, la compuerta AND. En este caso necesitaremos tres qubits: los dos que servirán de input, y un tercero donde almacenaremos el output. 

El primer paso es recordar cual es el funcionamiento de la compuerta AND:

| input 1 | input 2 | output |
| :---: | :---: | :---: |
|0 | 0 | 0 |
|0 | 1 | 0 |
| 1 | 0 | 0 |
| 1 | 1 | 1 |

Como puedes ver, esto es tan sencillo como implementar una compuerta Toffoli

In [ ]:
inputs = ["00","01","10","11"]

for estado in inputs:

    qc = QuantumCircuit(3,3)
    
    if int(estado[0]) == 1:
        qc.x(2)
    if int(estado[1]) == 1:
        qc.x(1)
 
    qc.ccx(1,2,0)
    
    qc.measure([0,1,2],[0,1,2])

    job = execute(qc,Aer.get_backend('qasm_simulator'),shots=100) 
    resultado = job.result().get_counts(qc)

    print("Input:",estado,"Output:",resultado)

---

Ahora, hagámos una compuerta un poco más complicada: la compuerta OR. En este caso, su funcionamiento es el siguiente:

| input 1 | input 2 | output |
| :---: | :---: | :---: |
|0 | 0 | 0 |
|0 | 1 | 1 |
| 1 | 0 | 1 |
| 1 | 1 | 1 |

Ahora, utilizando el mismo orden de bits de la tabla anterior, veamos cómo debe cambiar el estado de nuestros tres qubits, recordando que el qubit que almacenará el output se inicializa en $0$ por defecto.

- $|000\rangle \rightarrow |000\rangle$
- $|010\rangle \rightarrow |011\rangle$
- $|100\rangle \rightarrow |101\rangle$
- $|110\rangle \rightarrow |111\rangle$

Notamos que dejamos a nuestros dos inputs en el mismo estado, y solo alteramos el estado de nuestro qubit que almacena el output. Sin embargo, esto no significa que no podamos alterar el estado de nuestros qubits que funcionan como el input, siempre y cuando los regresemos a su valor original. Y eso es justamente lo que vamos a hacer.

Notamos que podemos aplicar una compuerta $X$ a los qubits 1 y 2, seguida de una compuerta CCNOT que utiliza a estos dos qubits como los qubits control. Esto nos permite cambiar el valor de nuestro output solo cuando nuestro estado inicial es $|000\rangle$

- $|000\rangle \rightarrow |110\rangle \rightarrow |111\rangle$
- $|010\rangle \rightarrow |100\rangle \rightarrow |100\rangle$
- $|100\rangle \rightarrow |010\rangle \rightarrow |010\rangle$
- $|110\rangle \rightarrow |000\rangle \rightarrow |000\rangle$

Ahora, solo queda aplicar una compuerta $X$ a nuestro qubit 0, seguido de otra compuerta $X$ a los qubits 1 y 2 para regresarlos a su estado inicial, y tendremos el resultado deseado.

- $|000\rangle \rightarrow |110\rangle \rightarrow |111\rangle \rightarrow |110\rangle \rightarrow |000\rangle$
- $|010\rangle \rightarrow |100\rangle \rightarrow |100\rangle \rightarrow |101\rangle \rightarrow |011\rangle$
- $|100\rangle \rightarrow |010\rangle \rightarrow |010\rangle \rightarrow |011\rangle \rightarrow |101\rangle$
- $|110\rangle \rightarrow |000\rangle \rightarrow |000\rangle \rightarrow |001\rangle \rightarrow |111\rangle$

Nuestra compuerta OR luce de la siguiente forma

<img src="Imagenes/OR.png" width="30%">

Ahora, vamos a codificar esta compuerta OR en una función, para poder aplicarla fácilmente.

In [ ]:
#Creamos una función con un parámetro: nuestro input
def OR(inp):
    qc = QuantumCircuit(3,3)

#Codificamos nuestro input en nuestro circuito
    if int(inp[0]) == 1:
        qc.x(2)
    if int(inp[1]) == 1:
        qc.x(1)
    
#Aplicamos la compuerta OR
    qc.x(1)
    qc.x(2)
    qc.ccx(1,2,0)
    qc.x(0)
    qc.x(1)
    qc.x(2)
    
#Hacemos una medición
    qc.measure([0,1,2],[0,1,2])
    
#Ejecutamos nuestro circuito
    job = execute(qc,Aer.get_backend('qasm_simulator'),shots=100)
    resultado = job.result().get_counts(qc)
    
#Imprimimos nuestro resultado
    print("Input:",inp,"Output:",resultado)

In [ ]:
inputs = ["00","01","10","11"]

for estado in inputs:
    OR(estado)

Como podemos ver, conseguimos justo lo que queríamos. Nuestro qubit del output es $0$ solo cuando ambos inputs son $0$. 

### Compuerta SWAP

Finalmente, vamos a ver la compuerta SWAP. El efecto de esta compuerta es bastante sencillo: intercambia el estado de dos qubits. Su sintaxis en Qiskit es `.swap(qubit1,qubit2)`. A diferencia de las compuertas controladas, en esta no importa el orden en que coloquen los qubits.

In [ ]:
inputs = ["00","01","10","11"]

for estado in inputs:

    qc = QuantumCircuit(2,2)
    
    if int(estado[0]) == 1:
        qc.x(1)
    if int(estado[1]) == 1:
        qc.x(0)
 
    qc.swap(1,0)
    
    qc.measure([0,1],[0,1])

    job = execute(qc,Aer.get_backend('qasm_simulator'),shots=100) 
    resultado = job.result().get_counts(qc)

    print("Input:",estado,"Output:",resultado)

Ahora, veamos como implementar esta compuerta utilizando solo compuertas de un qubit y compuertas CNOT. <br>
En este caso, solo necesitamos tres compuertas CNOT. Te dejamos de tarea realizar el proceso con cada uno de los $4$ estados base para convencerte de que estas tres compuertas CNOT intercambian los estados de los qubits. La descomposición de la compuerta SWAP en tres compuertas CNOT luce así:

<img src="Imagenes/SWAP.png" width="30%">

In [ ]:
def SWAP(inp):
    qc = QuantumCircuit(2,2)
    
    if int(inp[0]) == 1:
        qc.x(1)
    if int(inp[1]) == 1:
        qc.x(0)

    qc.cx(0,1)
    qc.cx(1,0)
    qc.cx(0,1)
    
    qc.measure([0,1],[0,1])

    job = execute(qc,Aer.get_backend('qasm_simulator'),shots=100) 
    resultado = job.result().get_counts(qc)

    print("Input:",estado,"Output:",resultado)

In [ ]:
inputs = ["00","01","10","11"]

for estado in inputs:
    SWAP(estado)